In [1]:
import dataIO

In [2]:
import lasagne
import theano
import theano.tensor as T
import numpy as np;
import time


# create Theano variables for input and target minibatch
sentence1 = T.ftensor3('s1')
sentence2 = T.ftensor3('s2')
mask1 = T.bmatrix('m1')
mask2 = T.bmatrix('m2')
target_var = T.ivector('ent')


Using gpu device 1: GeForce GTX 780 (CNMeM is enabled with initial size: 90.0% of memory, cuDNN 4007)
/home/xiaonan/anaconda2/lib/python2.7/site-packages/theano/tensor/signal/downsample.py:6: UserWarning: downsample module has been moved to the theano.tensor.signal.pool module.
  "downsample module has been moved to the theano.tensor.signal.pool module.")


In [4]:
# create a small convolutional neural network
from lasagne.nonlinearities import leaky_rectify, softmax
from lasagne.layers import InputLayer, Conv2DLayer, Pool2DLayer, LSTMLayer, ConcatLayer, DenseLayer, dropout, Conv1DLayer, ReshapeLayer

from const import *


def createNeuralNetwork():
    #(batchsize, sequence length, onehot vector length)
    in1 = InputLayer((None, None, kNUM_CHARS), sentence1)
    in2 = InputLayer((None, None, kNUM_CHARS), sentence2)
    l_mask1=lasagne.layers.InputLayer((None,None), mask1)
    l_mask2=lasagne.layers.InputLayer((None,None), mask2)

    num_LSTM_output = (512)
    lstm1_f = LSTMLayer(in1, num_LSTM_output,
        forgetgate=lasagne.layers.Gate(),
        nonlinearity=lasagne.nonlinearities.tanh,
        cell_init=lasagne.init.Constant(0.),
        hid_init=lasagne.init.Constant(0.), grad_clipping=kGRAD_CLIP,
        backwards=False,
        mask_input=l_mask1,
        only_return_final=True)
    lstm1_b = LSTMLayer(in1, num_LSTM_output,
        forgetgate=lasagne.layers.Gate(),
        nonlinearity=lasagne.nonlinearities.tanh,
        cell_init=lasagne.init.Constant(0.),
        hid_init=lasagne.init.Constant(0.), grad_clipping=kGRAD_CLIP,
        backwards=True,
        mask_input=l_mask1,
        only_return_final=True)

    lstm2_f = LSTMLayer(in2, num_LSTM_output,
        forgetgate=lasagne.layers.Gate(),
        nonlinearity=lasagne.nonlinearities.tanh,
        cell_init=lasagne.init.Constant(0.),
        hid_init=lasagne.init.Constant(0.), grad_clipping=kGRAD_CLIP,
        backwards=False,
        mask_input=l_mask2,
        only_return_final=True)
    lstm2_b = LSTMLayer(in2, num_LSTM_output,
        forgetgate=lasagne.layers.Gate(),
        nonlinearity=lasagne.nonlinearities.tanh,
        cell_init=lasagne.init.Constant(0.),
        hid_init=lasagne.init.Constant(0.), grad_clipping=kGRAD_CLIP,
        backwards=True,
        mask_input=l_mask2,
        only_return_final=True)

    network = ConcatLayer([lstm1_f, lstm1_b, lstm2_f, lstm2_b], axis=1) #(NONE-sentencesize by 2048)
    network = ReshapeLayer(network, (-1, 1, 4, num_LSTM_output));
    #(None by 4x512) I think.
    network = Conv2DLayer(network, 20, (3,3), pad='same',
                                         nonlinearity=leaky_rectify)
    #(20 by 4 by 52)
    network = Conv2DLayer(network, 10, (3,3), pad='same',
                                         nonlinearity=leaky_rectify)

    network = lasagne.layers.MaxPool2DLayer(network, (4,4), stride=2)

    network = DenseLayer(dropout(network, 0.5),
                                        128, nonlinearity=leaky_rectify,
                                        W=lasagne.init.Orthogonal())

    network = DenseLayer(dropout(network, 0.5), 3, nonlinearity=softmax)

    return network;


network = createNeuralNetwork();


In [5]:
def loadDriverModelFromFile(filename):
    print "Loading Neural Network Values from File"
    _v = np.load(filename)['model']
    lasagne.layers.set_all_param_values(network, _v)
    if type(filename)==str:
        print "LOADED"
        return;
    print "Loaded!"

print "HOLY SHIT IT COMPILED"
# create loss function
prediction = lasagne.layers.get_output(network)
print "HOLY SHIT IT COMPILED OUTPUT"

loss = lasagne.objectives.categorical_crossentropy(prediction, target_var)
loss = loss.mean() + 1e-4 * lasagne.regularization.regularize_network_params(
        network, lasagne.regularization.l2)
print "HOLY SHIT IT COMPILED LOSS FUNCTION WHAT"

# create parameter update expressions
params = lasagne.layers.get_all_params(network, trainable=True)
updates = lasagne.updates.nesterov_momentum(loss, params, learning_rate=0.01,
                                            momentum=0.9)
print "HOLY SHIT IT COMPILED UPDATES"

# compile training function that updates parameters and returns training loss
train_fn = theano.function([sentence1, sentence2, mask1, mask2, target_var], loss, updates=updates, allow_input_downcast=True)
# train_fn = theano.function([sentence1, sentence2, target_var], loss, updates=updates)
print "HOLY SHIT IT COMPILED TRAINING FUNCTION"



HOLY SHIT IT COMPILED
HOLY SHIT IT COMPILED OUTPUT
HOLY SHIT IT COMPILED LOSS FUNCTION WHAT
HOLY SHIT IT COMPILED UPDATES
HOLY SHIT IT COMPILED TRAINING FUNCTION


In [10]:
# train network (assuming you've got some training data in numpy arrays)

def makeMask(batch): #list of matricies of variable size.
    m = np.zeros((len(batch), max([b.shape[0] for b in batch])))
    for i,b in enumerate(batch):
        m[i, 0:b.shape[0]] = 1.0
    return m;

for epoch in range(kNUM_EPOCHS):
    batchCount = 0
    loss = 0
    print "HOLY SHIT IT EPOCHS!"
    #for (batch1,m1), (batch2,m2), ys in dataIO.readChunk(kBATCH_SIZE, './snli_1.0/snli_1.0_train.txt'):
    for (batch1,m1), (batch2,m2), ys in dataIO.readChunk(kBATCH_SIZE, './snli_1.0/snli_1.0_train.txt'):
        #         if batchCount >= kMAX_BATCHES:
        #             break
        # import code
        # code.interact(local=locals())
        # print batch1.shape, batch2.shape, ys.shape
        # loss += train_fn(batch1, batch2, makeMask(batch1), makeMask(batch2), ys)
        loss += train_fn(batch1, batch2, m1, m2, ys)
        batchCount += 1

    print("Epoch %d: Loss %g" % (epoch + 1, loss / batchCount))
   
    vals = lasagne.layers.get_all_param_values(network)
    modelFile = open("modelStore/"+time.strftime("%m%d-%H%M%S")+".pkl", mode="w")    
    np.savez(modelFile, model=vals)
    print ">>>", modelFile.name
    modelFile.close()
    print "Done."


HOLY SHIT IT EPOCHS!
reading 
Epoch 1: Loss 1.12902
>>> modelStore/0602-020915.pkl
Done.
HOLY SHIT IT EPOCHS!
reading 
Epoch 2: Loss 1.0996
>>> modelStore/0602-125926.pkl
Done.
HOLY SHIT IT EPOCHS!
reading 


KeyboardInterrupt: 

In [11]:
vals = lasagne.layers.get_all_param_values(network)
modelFile = open("modelStore/"+time.strftime("%m%d-%H%M%S")+".pkl", mode="w")    
np.savez(modelFile, model=vals)
print ">>>", modelFile.name
modelFile.close()
print "Done."


>>> modelStore/0602-165522.pkl
Done.


In [12]:
# use trained network for predictions
test_prediction = lasagne.layers.get_output(network, deterministic=True)
predict_fn = theano.function([sentence1, sentence2, mask1, mask2], T.argmax(test_prediction, axis=1))

In [14]:
# Save model
# Check on dev set
batch_num = 0
total_classifications = 0
total_correct = 0
total_incorrect = 0
for (batch1,m1), (batch2,m2), ys in dataIO.readChunk(kBATCH_SIZE, './snli_1.0/snli_1.0_dev.txt'):
    results = predict_fn(batch1, batch2, m1, m2);
    for i in range(len(results)):
        if results[i] == ys[i]:
            total_correct += 1
        else:
            total_incorrect += 1
        total_classifications += 1
    batch_num += 1
    print batch_num, "\r", 
    #if batch_num == 1666: break


reading 


KeyboardInterrupt: 

In [15]:
# Stats print out

print "Total classifications: " + str(total_classifications)
print "Total correct: " + str(total_correct)
print "Total incorrect: " + str(total_incorrect)

Total classifications: 2394
Total correct: 847
Total incorrect: 1547


In [59]:
# Save model
# Check on test set
batch_num = 0
total_classifications = 0
total_correct = 0
total_incorrect = 0
for (batch1,m1), (batch2,m2), ys in dataIO.readChunk(kBATCH_SIZE, './snli_1.0/snli_1.0_train.txt'):
    if batch_num == 500:
        break
    results = predict_fn(batch1, batch2, m1, m2)
    for i in range(len(results)):
        if results[i] == ys[i]:
            total_correct += 1
        else:
            total_incorrect += 1
        total_classifications += 1
    batch_num += 1

In [21]:
# Stats print out for train set

print "Total classifications: " + str(total_classifications)
print "Total correct: " + str(total_correct)
print "Total incorrect: " + str(total_incorrect)

Total classifications: 600
Total correct: 199
Total incorrect: 401
